## 05_05: Introduction to parallel programming

In [1]:
import multiprocessing

In [2]:
import numpy as np

import matplotlib
import matplotlib.pyplot as pp

In [3]:
%%file mandel.py

import numpy as np

def compute_mandel(c, maxit=256):
    z = 0.0j

    for it in range(1, maxit):
        z = z*z + c
        
        if abs(z) > 2.0:
            return it

    return np.inf

Writing mandel.py


In [4]:
import mandel

In [5]:
def run_mandel(extent=(-2.0,1.0,-1.5,1.5), res=256, maxit=256):
    xs = np.linspace(extent[0], extent[1], res)
    ys = np.linspace(extent[2], extent[3], res)
    
    escaped = np.zeros((res,res), np.float64)
    
    for i in range(res):
        for j in range(res):
            c = xs[i] + 1j * ys[j]
            escaped[i,j] = mandel.compute_mandel(c, maxit)
    
    return escaped / maxit

In [6]:
%timeit run_mandel()

1.61 s ± 6.78 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
def run_mandel_multiproc(extent=(-2.0,1.0,-1.5,1.5), res=256, maxit=256, nproc=4):
    xs = np.linspace(extent[0], extent[1], res)
    ys = np.linspace(extent[2], extent[3], res)
    
    escaped = np.zeros((res,res), np.float64)
    
    # create the pool of nproc processes
    pool = multiprocessing.Pool(processes=nproc)
    
    for i in range(res):
        # feed the elements of the iterator to compute_mandel,
        # collect results into row of escaped
        escaped[i,:] = pool.map(mandel.compute_mandel, (xs[i] + 1j * ys[j] for j in range(res)))
        
    # close the pool and free the processes
    pool.close()
    
    return escaped / maxit

In [8]:
%timeit run_mandel_multiproc()

907 ms ± 7.11 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
%%file mandelvec.py

import numpy as np
import mandel

def compute_mandel_vec(cs, maxit=256):
    return np.array([mandel.compute_mandel(c, maxit) for c in cs])

Writing mandelvec.py


In [10]:
import mandelvec

In [11]:
def run_mandel_multiproc2(extent=(-2.0,1.0,-1.5,1.5), res=256, maxit=256, nproc=4):
    xs = np.linspace(extent[0], extent[1], res)
    ys = np.linspace(extent[2], extent[3], res)
    
    escaped = np.zeros((res,res), np.float64)
    
    pool = multiprocessing.Pool(processes=nproc)

    rows = pool.map(mandelvec.compute_mandel_vec,
                    # compute coordinates along entire row, feed it to compute_mandel_vec 
                    (xs[i] + 1j * ys[:] for i in range(res)))
            
    pool.close()
    
    return np.vstack(rows) / maxit

In [12]:
%timeit run_mandel_multiproc2()

421 ms ± 5.11 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
